# Proyecto II: Generación automatica de texto

En este poryecto el objetivo es desarrollar un modelo de generación aunotática de texto utilizando redes recurrentes.Para este modelo deben realizar los siguientes pasos:

1. Importar un corpus (texto con el que van a entrenar el modelo). Este corpus debe ser grande y contener mas de 100 mil palabras. Para este proyecto se pedira que entrenen con dos corpus de estilo diferente. Por ejemplo, un corpus con las obras de gabriel garcia marquez, otro corpus con poemas.
2. Preprocesar el texto. Tenga en cuenta que la entrada debe ser lo más homogenea posible, para poder entrenar las redes. Algunos modelos eliminan la puntuación, sin embargo, si esto se hace el texto generado no tendra puntuación. Yo recomiendo dejar estos elementos. Al igual que los saltos de lineas si entrenan poemas, o incluso en el texto para generar el punto aparte.
3. Preparar los datos de entrenamiento, esto inclutye los wordembeddings, o la generación de indices que puedan ser utilziados en un acapa de embedding.
4. Diseño de la arquitectura de red. Yo aconsejo utilizar un acapa de embeddings, para que el algoritmo por si solo aprenda el embedding apropiado para la tarea.
5. generación de texto. Tenga en cuenta que al entrenar se entrena en modo profesor, pero al generar se debe retroalimentar la salida del modelo como entrada para producir las palabras siguientes. Para parar su ejecución pueden generar un token de finalizacion /<end/> y solo parar cuando este token aparezca, si este es el caso ese token debe estar en el diccionario y en el texto que esten procesando. Otra forma es generar x número de palabras y parar la ejecución una vez se haya cumplido con ese número de palabras.
6. Para ejecutar el programa deben proporcionar una semilla, esta semilla debe contener palabras que esten en el vocabulario con el que se entreno, una vez hecho esto se puede correr el modelo.
7. En la entrega deben especificar paso a paso el proceso realizado, comparar los resultados del texto generado con ambos modelos entrenados (uno para cada corpus) y discutir a profundidad los resultados obtenidos, problemas y posibles mejoras.

Dado que este proyecto requiere el uso de temas no vistos a profundidad en clase, les proporciono el siguiente blog que pueden usar como base para el desarrollo del proyecto. Les recomiendo lo lean, lo sigan paso a paso, ejecuten el código que allí se encuentra y luego hagan las modificaciones necesarias para el desarrollo de su proyecto. El blog lo pueden encontrar en este [link](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/). 

El proyecto lo deben entregar el Lunes 24 de abril


# Desarrollo

Como primer paso, se definen cuatro funciones. La primera función carga y lee los archivos de texto y retorna una única lista. La segunda función, se encarga de limpiar el texto, eliminando algunos caracteres especiales y castear (casting) las palabras a minúscula, esta función retorna una lista con todas las palabras de acuerdo con el orden que fueron escritas. La tercera función se encarga de guardar el texto ya procesado en un nuevo archivo de texto. Por último, la cuarta función recibe las palabras (tokens) y una longitud definida, dicha longitud define el tamaño de la secuencia que se generara a partir de las palabras de entrada. Estas cuatro funciones componen todo el preprocesamiento de texto para cada corpus

In [4]:
def load_docs(filenames):
    complete_text = []
    for filename in filenames:
        file = open(filename, 'r')
        text = file.read()
        complete_text += text
        file.close()
    return text

def clean_text(doc):
    doc = doc.replace('--', '')
    doc = doc.replace('—', '')
    doc = doc.replace('-', '')
    doc = doc.replace('«', '')
    doc = doc.replace('»', '')
    doc = doc.replace('.', '. ')
    doc = doc.replace(' .', '.')
    doc = doc.replace('[', '')
    doc = doc.replace(']', '')
    doc = doc.replace('...', '')
    doc = doc.replace(' !', '!')
    doc = doc.replace(' ?', '?')
    doc = doc.replace(' ,', ',')
    doc = doc.replace('( ', '(')
    doc = doc.replace(' )', ')')
    tokens = doc.split()
    tokens = [token.lower() for token in tokens if token != '']
    print("Clean text completed")
    print(f'Total tokens: {len(tokens)}')
    print(f'Unique tokens: {len(set(tokens))}')
    return tokens

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

def to_sequences(length, tokens):
    sequences = []
    for i in range(length, len(tokens)):
        seq = tokens[i-length:i]
        if len(seq) == length:
            line = ' '.join(seq)
            sequences.append(line)
        else:
            sequences
    print("Sequences formed")
    print(f'Total Sequences: {len(sequences)}')
    return sequences

In [5]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

## Corpus 1 - Primer Modelo
Como primer corpus, se consideró la saga completa de Harry Potter. Se comienza cargando los siete libros de Harry Potter y se realiza el preprocesamiento. Observe que se cuenta con 208.782 palabras 

In [2]:
in_filenames = ['Books_data/HarryPotter/J.K. Rowling - Harry Potter 1 - La Piedra Filosofal.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 2 - La Cámara Secreta.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 4 - El Cáliz de Fuego.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 5 - La Orden del Fenix.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 6 - El Misterio del Príncipe.txt',
               'Books_data/HarryPotter/J.K. Rowling - Harry Potter 7 - Las Reliquias de la Muerte.txt']
doc = load_docs(in_filenames)
tokens = clean_text(doc)
lines = to_sequences(51, tokens)

out_filename = 'HarryPoter_CleanText.txt'
save_doc(lines, out_filename)

Clean text completed
Total tokens: 208782
Unique tokens: 26661
Sequences formed
Total Sequences: 208731


Ahora, se configura el uso de GPU para acelerar el tiempo de ejecución (entrenamiento) del modelo.

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Se carga el archivo de texto que contiene el texto ya procesado y se realiza la tokenizacion del mismo. También, se definen las secuencias de acuerdo con los tokens definidos (índices). 

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from numpy import array
from pickle import dump

in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
sequences = [sequence for sequence in sequences if len(sequence) == 51]
vocab_size = len(tokenizer.word_index) + 1

Se define el modelo el cual consta de una capa de embedding, dos capas de Long short-term memory (LSTM) y 2 capas densas, la primera con una función de activación relu y la segunda con una función de activación softmax. Como optimizador se utiliza Adam.

In [3]:
# define model
seq_length = 50 #X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(80, return_sequences=True))
model.add(LSTM(80))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            886050    
                                                                 
 lstm (LSTM)                 (None, 50, 80)            41920     
                                                                 
 lstm_1 (LSTM)               (None, 80)                51520     
                                                                 
 dense (Dense)               (None, 50)                4050      
                                                                 
 dense_1 (Dense)             (None, 17721)             903771    
                                                                 
Total params: 1,887,311
Trainable params: 1,887,311
Non-trainable params: 0
_________________________________________________________________
None


Dado el tamaño de los datos, la arquitectura del modelo y la capacidad computacional disponible, los datos son divididos en 4 secciones y cada sección se presenta al modelo con un batch size de 128 y 40 épocas. Los resultados del entrenamiento son almacenados en los archivos `model_HarryPotter.h5` y `tokenizer_HarryPotter.pkl`. 

In [4]:
size_data = len(sequences)//4

for i in range(size_data,len(sequences)+1, size_data):
    sub_sequences = sequences[i-size_data:i]
    sequences_aux = array(sub_sequences)
    
    X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
    y = to_categorical(y, num_classes=vocab_size)

    model.fit(X, y, batch_size=128, epochs=40)
    
# save the model to file
model.save('model_HarryPotter.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_HarryPotter.pkl', 'wb'))

2022-04-25 05:51:43.226467: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-25 05:51:47.641404: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 10s 15ms/step - loss: 7.3237 - accuracy: 0.0460
Epoch 2/40
408/408 [==============================] - 6s 15ms/step - loss: 6.7363 - accuracy: 0.0481
Epoch 3/40
408/408 [==============================] - 6s 15ms/step - loss: 6.4804 - accuracy: 0.0622
Epoch 4/40
408/408 [==============================] - 6s 15ms/step - loss: 6.3168 - accuracy: 0.0681
Epoch 5/40
408/408 [==============================] - 6s 15ms/step - loss: 6.1756 - accuracy: 0.0766
Epoch 6/40
408/408 [==============================] - 6s 15ms/step - loss: 6.0242 - accuracy: 0.0905
Epoch 7/40
408/408 [==============================] - 6s 15ms/step - loss: 5.8908 - accuracy: 0.1039
Epoch 8/40
408/408 [==============================] - 6s 15ms/step - loss: 5.7773 - accuracy: 0.1114
Epoch 9/40
408/408 [==============================] - 6s 15ms/step - loss: 5.6762 - accuracy: 0.1192
Epoch 10/40
408/408 [==============================] - 6s 15ms/step - loss: 5.5797 - accur

2022-04-25 05:56:00.605752: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-25 05:56:04.979942: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 6s 15ms/step - loss: 7.1399 - accuracy: 0.0683
Epoch 2/40
408/408 [==============================] - 6s 15ms/step - loss: 6.2375 - accuracy: 0.0851
Epoch 3/40
408/408 [==============================] - 6s 15ms/step - loss: 6.0628 - accuracy: 0.0946
Epoch 4/40
408/408 [==============================] - 6s 15ms/step - loss: 5.9292 - accuracy: 0.1058
Epoch 5/40
408/408 [==============================] - 6s 15ms/step - loss: 5.8062 - accuracy: 0.1126
Epoch 6/40
408/408 [==============================] - 6s 15ms/step - loss: 5.6930 - accuracy: 0.1201
Epoch 7/40
408/408 [==============================] - 6s 15ms/step - loss: 5.5824 - accuracy: 0.1253
Epoch 8/40
408/408 [==============================] - 6s 15ms/step - loss: 5.4786 - accuracy: 0.1306
Epoch 9/40
408/408 [==============================] - 6s 15ms/step - loss: 5.3759 - accuracy: 0.1369
Epoch 10/40
408/408 [==============================] - 6s 15ms/step - loss: 5.2786 - accura

2022-04-25 06:00:14.661106: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 6s 15ms/step - loss: 7.4634 - accuracy: 0.0597
Epoch 2/40
408/408 [==============================] - 6s 15ms/step - loss: 6.5568 - accuracy: 0.0858
Epoch 3/40
408/408 [==============================] - 6s 15ms/step - loss: 6.2718 - accuracy: 0.0980
Epoch 4/40
408/408 [==============================] - 6s 15ms/step - loss: 6.0812 - accuracy: 0.1047
Epoch 5/40
408/408 [==============================] - 6s 15ms/step - loss: 5.9322 - accuracy: 0.1135
Epoch 6/40
408/408 [==============================] - 6s 15ms/step - loss: 5.8036 - accuracy: 0.1201
Epoch 7/40
408/408 [==============================] - 6s 15ms/step - loss: 5.6877 - accuracy: 0.1253
Epoch 8/40
408/408 [==============================] - 6s 15ms/step - loss: 5.5819 - accuracy: 0.1297
Epoch 9/40
408/408 [==============================] - 6s 15ms/step - loss: 5.4822 - accuracy: 0.1354
Epoch 10/40
408/408 [==============================] - 6s 15ms/step - loss: 5.3911 - accura

## Corpus 1 - Segundo Modelo

In [3]:
# define model
seq_length = 50 #X.shape[1]

model3 = Sequential()
model3.add(Embedding(vocab_size, 50, input_length=seq_length))
model3.add(LSTM(20, return_sequences=True))
model3.add(LSTM(20))
model3.add(Dense(30, activation='relu'))
model3.add(Dense(vocab_size, activation='softmax'))
print(model3.summary())
# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            886050    
                                                                 
 lstm (LSTM)                 (None, 50, 20)            5680      
                                                                 
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 30)                630       
                                                                 
 dense_1 (Dense)             (None, 17721)             549351    
                                                                 
Total params: 1,444,991
Trainable params: 1,444,991
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
size_data = len(sequences)//4

for i in range(size_data,len(sequences)+1, size_data):
    sub_sequences = sequences[i-size_data:i]
    sequences_aux = array(sub_sequences)
    
    X, y = sequences_aux[:,:-1], sequences_aux[:,-1]
    y = to_categorical(y, num_classes=vocab_size)

    model3.fit(X, y, batch_size=128, epochs=40)
    
# save the model to file
model3.save('model2_HarryPotter.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer2_HarryPotter.pkl', 'wb'))

2022-04-25 14:27:45.196313: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-25 14:27:49.631517: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 10s 14ms/step - loss: 7.4237 - accuracy: 0.0432
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.7387 - accuracy: 0.0463
Epoch 3/40
408/408 [==============================] - 6s 14ms/step - loss: 6.6442 - accuracy: 0.0490
Epoch 4/40
408/408 [==============================] - 6s 14ms/step - loss: 6.5204 - accuracy: 0.0612
Epoch 5/40
408/408 [==============================] - 6s 14ms/step - loss: 6.3776 - accuracy: 0.0662
Epoch 6/40
408/408 [==============================] - 6s 14ms/step - loss: 6.2511 - accuracy: 0.0693
Epoch 7/40
408/408 [==============================] - 6s 14ms/step - loss: 6.1342 - accuracy: 0.0731
Epoch 8/40
408/408 [==============================] - 6s 14ms/step - loss: 6.0489 - accuracy: 0.0742
Epoch 9/40
408/408 [==============================] - 6s 14ms/step - loss: 5.9795 - accuracy: 0.0792
Epoch 10/40
408/408 [==============================] - 6s 14ms/step - loss: 5.9151 - accur

2022-04-25 14:31:35.767323: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.
2022-04-25 14:31:40.157860: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 5s 13ms/step - loss: 7.2226 - accuracy: 0.0613
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.2547 - accuracy: 0.0826
Epoch 3/40
408/408 [==============================] - 5s 13ms/step - loss: 6.0671 - accuracy: 0.0921
Epoch 4/40
408/408 [==============================] - 5s 13ms/step - loss: 5.9107 - accuracy: 0.1024
Epoch 5/40
408/408 [==============================] - 5s 13ms/step - loss: 5.7693 - accuracy: 0.1113
Epoch 6/40
408/408 [==============================] - 5s 13ms/step - loss: 5.6398 - accuracy: 0.1208
Epoch 7/40
408/408 [==============================] - 5s 13ms/step - loss: 5.5148 - accuracy: 0.1290
Epoch 8/40
408/408 [==============================] - 5s 13ms/step - loss: 5.3956 - accuracy: 0.1376
Epoch 9/40
408/408 [==============================] - 5s 13ms/step - loss: 5.2824 - accuracy: 0.1457
Epoch 10/40
408/408 [==============================] - 5s 13ms/step - loss: 5.1720 - accura

2022-04-25 14:35:20.829574: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3697096788 exceeds 10% of free system memory.


Epoch 1/40
408/408 [==============================] - 5s 13ms/step - loss: 7.7507 - accuracy: 0.0515
Epoch 2/40
408/408 [==============================] - 5s 13ms/step - loss: 6.6059 - accuracy: 0.0806
Epoch 3/40
408/408 [==============================] - 5s 13ms/step - loss: 6.2691 - accuracy: 0.0919
Epoch 4/40
408/408 [==============================] - 6s 14ms/step - loss: 6.0496 - accuracy: 0.1003
Epoch 5/40
408/408 [==============================] - 5s 13ms/step - loss: 5.8799 - accuracy: 0.1076
Epoch 6/40
408/408 [==============================] - 6s 14ms/step - loss: 5.7289 - accuracy: 0.1147
Epoch 7/40
408/408 [==============================] - 5s 13ms/step - loss: 5.5914 - accuracy: 0.1218
Epoch 8/40
408/408 [==============================] - 5s 13ms/step - loss: 5.4638 - accuracy: 0.1287
Epoch 9/40
408/408 [==============================] - 5s 13ms/step - loss: 5.3418 - accuracy: 0.1363
Epoch 10/40
408/408 [==============================] - 5s 13ms/step - loss: 5.2269 - accura

## Corpus 2 - Primer Modelo

Para el segundo modelo, se consideró un corpus con el libro El Código Da Vinci. De igual forma al caso anterior, comenzamos cargando el texto y realizando el debido preprocesamiento para así limpiar los datos. Observe que, para este caso, se tienen 153.890 palabras, lo cual es una cuarta parte menos al caso anterior.

In [6]:
in_filename = 'Books_data/El Codigo Da Vinci.txt'
doc = load_doc(in_filename)
tokens = clean_text(doc)
lines = to_sequences(51, tokens)

out_filename = 'CodigoDaVinci_CleanText.txt'
save_doc(lines, out_filename)

Clean text completed
Total tokens: 153890
Unique tokens: 22182
Sequences formed
Total Sequences: 153839


Antes de correr el modelo, configuramos el uso de la GPU. 

In [7]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2022-04-26 21:40:57.177338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 21:40:57.177374: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-26 21:41:01.315944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-26 21:41:01.315987: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-26 21:41:01.316017: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist


IndexError: list index out of range

Cargamos el archivo que contiene los datos limpios, se realiza la tokenizacion y se crean las secuencias. 

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from numpy import array
from pickle import dump


in_filename = 'CodigoDaVinci_CleanText.txt'
doc2 = load_doc(in_filename)
lines2 = doc2.split('\n')

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(lines2)
sequences2 = tokenizer2.texts_to_sequences(lines2)
sequences2 = [sequence for sequence in sequences2 if len(sequence) == 51]
vocab_size2 = len(tokenizer2.word_index) + 1

Se define el modelo. Observe que el modelo es igual al anterior. 

In [3]:
# define model
seq_length2 = 50 #X.shape[1]

model2 = Sequential()
model2.add(Embedding(vocab_size2, 50, input_length=seq_length2))
model2.add(LSTM(80, return_sequences=True))
model2.add(LSTM(80))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(vocab_size2, activation='softmax'))
print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            774550    
                                                                 
 lstm (LSTM)                 (None, 50, 80)            41920     
                                                                 
 lstm_1 (LSTM)               (None, 80)                51520     
                                                                 
 dense (Dense)               (None, 50)                4050      
                                                                 
 dense_1 (Dense)             (None, 15491)             790041    
                                                                 
Total params: 1,662,081
Trainable params: 1,662,081
Non-trainable params: 0
_________________________________________________________________
None


Para el caso de este corpus, dado que es más pequeño que el primero, los datos se dividieron en 3 secciones y, de igual manera, se entró con un batch size de 128 con la diferencia de que el entrenamiento se realiza por 50 épocas. Los resultados del modelo se almacenan en los archivos `model_DaVinci.h5` y `tokenizer_DaVinci.pkl` 

In [4]:
size_data = len(sequences2)//3

for i in range(size_data,len(sequences2)+1, size_data):
    sub_sequences2 = sequences2[i-size_data:i]
    sequences_aux2 = array(sub_sequences2)
    
    X, y = sequences_aux2[:,:-1], sequences_aux2[:,-1]
    y = to_categorical(y, num_classes=vocab_size2)

    model2.fit(X, y, batch_size=128, epochs=50)
    
# save the model to file
model2.save('model_DaVinci.h5')
# save the tokenizer
dump(tokenizer2, open('tokenizer_DaVinci.pkl', 'wb'))

2022-04-25 06:22:06.160705: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-25 06:22:09.421024: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 9s 14ms/step - loss: 7.2735 - accuracy: 0.0508
Epoch 2/50
342/342 [==============================] - 5s 14ms/step - loss: 6.6792 - accuracy: 0.0541
Epoch 3/50
342/342 [==============================] - 5s 14ms/step - loss: 6.4477 - accuracy: 0.0726
Epoch 4/50
342/342 [==============================] - 5s 14ms/step - loss: 6.2327 - accuracy: 0.0801
Epoch 5/50
342/342 [==============================] - 5s 14ms/step - loss: 6.1081 - accuracy: 0.0842
Epoch 6/50
342/342 [==============================] - 5s 14ms/step - loss: 5.9814 - accuracy: 0.0895
Epoch 7/50
342/342 [==============================] - 5s 14ms/step - loss: 5.8410 - accuracy: 0.1011
Epoch 8/50
342/342 [==============================] - 5s 14ms/step - loss: 5.7174 - accuracy: 0.1119
Epoch 9/50
342/342 [==============================] - 5s 14ms/step - loss: 5.6091 - accuracy: 0.1172
Epoch 10/50
342/342 [==============================] - 5s 14ms/step - loss: 5.5104 - accura

2022-04-25 06:26:20.128854: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-25 06:26:23.374421: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 5s 14ms/step - loss: 7.6973 - accuracy: 0.0568
Epoch 2/50
342/342 [==============================] - 5s 14ms/step - loss: 6.5993 - accuracy: 0.0748
Epoch 3/50
342/342 [==============================] - 5s 14ms/step - loss: 6.3681 - accuracy: 0.0885
Epoch 4/50
342/342 [==============================] - 5s 14ms/step - loss: 6.2036 - accuracy: 0.0920
Epoch 5/50
342/342 [==============================] - 5s 14ms/step - loss: 6.0630 - accuracy: 0.0981
Epoch 6/50
342/342 [==============================] - 5s 14ms/step - loss: 5.9411 - accuracy: 0.1061
Epoch 7/50
342/342 [==============================] - 5s 14ms/step - loss: 5.8288 - accuracy: 0.1124
Epoch 8/50
342/342 [==============================] - 5s 14ms/step - loss: 5.7178 - accuracy: 0.1199
Epoch 9/50
342/342 [==============================] - 5s 14ms/step - loss: 5.6139 - accuracy: 0.1255
Epoch 10/50
342/342 [==============================] - 5s 14ms/step - loss: 5.5132 - accura

2022-04-25 06:30:30.681244: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 5s 14ms/step - loss: 7.5849 - accuracy: 0.0503
Epoch 2/50
342/342 [==============================] - 5s 14ms/step - loss: 6.8470 - accuracy: 0.0711
Epoch 3/50
342/342 [==============================] - 5s 14ms/step - loss: 6.6282 - accuracy: 0.0798
Epoch 4/50
342/342 [==============================] - 5s 14ms/step - loss: 6.4752 - accuracy: 0.0845
Epoch 5/50
342/342 [==============================] - 5s 14ms/step - loss: 6.3478 - accuracy: 0.0889
Epoch 6/50
342/342 [==============================] - 5s 14ms/step - loss: 6.2348 - accuracy: 0.0938
Epoch 7/50
342/342 [==============================] - 5s 14ms/step - loss: 6.1286 - accuracy: 0.1010
Epoch 8/50
342/342 [==============================] - 5s 14ms/step - loss: 6.0256 - accuracy: 0.1084
Epoch 9/50
342/342 [==============================] - 5s 14ms/step - loss: 5.9323 - accuracy: 0.1152
Epoch 10/50
342/342 [==============================] - 5s 14ms/step - loss: 5.8348 - accura

NameError: name 'model' is not defined

## Corpus 2 - Segundo Modelo

In [4]:
# define model
seq_length2 = 50 #X.shape[1]

model4 = Sequential()
model4.add(Embedding(vocab_size2, 50, input_length=seq_length2))
model4.add(LSTM(80, return_sequences=True))
model4.add(LSTM(65, return_sequences=True))
model4.add(LSTM(50))
model4.add(Dense(50, activation='relu'))
model4.add(Dense(vocab_size2, activation='softmax'))
print(model4.summary())
# compile model
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            774550    
                                                                 
 lstm_2 (LSTM)               (None, 50, 80)            41920     
                                                                 
 lstm_3 (LSTM)               (None, 50, 65)            37960     
                                                                 
 lstm_4 (LSTM)               (None, 50)                23200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 15491)             790041    
                                                                 
Total params: 1,670,221
Trainable params: 1,670,221
No

In [5]:
size_data = len(sequences2)//3

for i in range(size_data,len(sequences2)+1, size_data):
    sub_sequences2 = sequences2[i-size_data:i]
    sequences_aux2 = array(sub_sequences2)
    
    X, y = sequences_aux2[:,:-1], sequences_aux2[:,-1]
    y = to_categorical(y, num_classes=vocab_size2)

    model4.fit(X, y, batch_size=128, epochs=50)
    
# save the model to file
model4.save('model2_DaVinci.h5')
# save the tokenizer
dump(tokenizer2, open('tokenizer2_DaVinci.pkl', 'wb'))

2022-04-25 07:18:13.370810: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-25 07:18:16.637312: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 11s 17ms/step - loss: 7.2868 - accuracy: 0.0519
Epoch 2/50
342/342 [==============================] - 6s 16ms/step - loss: 6.7698 - accuracy: 0.0541
Epoch 3/50
342/342 [==============================] - 6s 16ms/step - loss: 6.7060 - accuracy: 0.0540
Epoch 4/50
342/342 [==============================] - 6s 16ms/step - loss: 6.5476 - accuracy: 0.0642
Epoch 5/50
342/342 [==============================] - 6s 16ms/step - loss: 6.3261 - accuracy: 0.0772
Epoch 6/50
342/342 [==============================] - 6s 16ms/step - loss: 6.1445 - accuracy: 0.0801
Epoch 7/50
342/342 [==============================] - 6s 17ms/step - loss: 6.0117 - accuracy: 0.0824
Epoch 8/50
342/342 [==============================] - 6s 16ms/step - loss: 5.9026 - accuracy: 0.0851
Epoch 9/50
342/342 [==============================] - 6s 16ms/step - loss: 5.8010 - accuracy: 0.0875
Epoch 10/50
342/342 [==============================] - 6s 17ms/step - loss: 5.7057 - accur

2022-04-25 07:23:06.554553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.
2022-04-25 07:23:09.801661: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 6s 17ms/step - loss: 7.1229 - accuracy: 0.0738
Epoch 2/50
342/342 [==============================] - 6s 17ms/step - loss: 6.2429 - accuracy: 0.0878
Epoch 3/50
342/342 [==============================] - 6s 17ms/step - loss: 6.0068 - accuracy: 0.0962
Epoch 4/50
342/342 [==============================] - 6s 17ms/step - loss: 5.8553 - accuracy: 0.1047
Epoch 5/50
342/342 [==============================] - 6s 16ms/step - loss: 5.7233 - accuracy: 0.1105
Epoch 6/50
342/342 [==============================] - 6s 17ms/step - loss: 5.6031 - accuracy: 0.1165
Epoch 7/50
342/342 [==============================] - 6s 17ms/step - loss: 5.4906 - accuracy: 0.1232
Epoch 8/50
342/342 [==============================] - 6s 17ms/step - loss: 5.3821 - accuracy: 0.1289
Epoch 9/50
342/342 [==============================] - 6s 17ms/step - loss: 5.2737 - accuracy: 0.1353
Epoch 10/50
342/342 [==============================] - 6s 17ms/step - loss: 5.1702 - accura

2022-04-25 07:27:56.795111: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2710367324 exceeds 10% of free system memory.


Epoch 1/50
342/342 [==============================] - 6s 17ms/step - loss: 7.5222 - accuracy: 0.0424
Epoch 2/50
342/342 [==============================] - 6s 17ms/step - loss: 6.5311 - accuracy: 0.0762
Epoch 3/50
342/342 [==============================] - 6s 17ms/step - loss: 6.3182 - accuracy: 0.0793
Epoch 4/50
342/342 [==============================] - 6s 17ms/step - loss: 6.1694 - accuracy: 0.0838
Epoch 5/50
342/342 [==============================] - 6s 17ms/step - loss: 6.0433 - accuracy: 0.0899
Epoch 6/50
342/342 [==============================] - 6s 17ms/step - loss: 5.9314 - accuracy: 0.0977
Epoch 7/50
342/342 [==============================] - 6s 17ms/step - loss: 5.9675 - accuracy: 0.1021
Epoch 8/50
342/342 [==============================] - 6s 17ms/step - loss: 5.8792 - accuracy: 0.1068
Epoch 9/50
342/342 [==============================] - 6s 17ms/step - loss: 5.7447 - accuracy: 0.1145
Epoch 10/50
342/342 [==============================] - 6s 17ms/step - loss: 5.6095 - accura

# Generación de texto

In [1]:
from random import randint
from pickle import load

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from numpy import argmax

In [2]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        predict_yhat = model.predict(encoded, verbose=0)
        yhat = argmax(predict_yhat, axis=1)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

## Corpus 1 - Modelo 1

In [3]:
# load cleaned text sequences
in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model_HarryPotter.h5')

# load the tokenizer
tokenizer = load(open('tokenizer_HarryPotter.pkl', 'rb'))

In [4]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

el muchacho sabía que no podían hacerle daño, miró hacia atrás, preguntándose qué o a quién esperaba aquel hombre de repente, un cegador e irregular chorro de luz blanca surcó el aire. harry pensó que era un rayo, pero snape se había arrodillado y la varita se le había caído de

la sala de piedra de los amigos se había llegado a la sala de piedra de los amigos se había llegado a la sala de piedra de los amigos se había llegado a la sala de piedra de los amigos se había llegado a la sala de piedra de los


## Corpus 1 - Modelo 2

In [5]:
# load cleaned text sequences
in_filename = 'HarryPoter_CleanText.txt'
doc = load_doc(in_filename)
lines2 = doc.split('\n')
seq_length2 = len(lines[0].split()) - 1

# load the model
model2 = load_model('model2_HarryPotter.h5')

# load the tokenizer
tokenizer2 = load(open('tokenizer2_HarryPotter.pkl', 'rb'))

In [6]:
# select a seed text
seed_text2 = lines2[randint(0,len(lines2))]
print(seed_text2 + '\n')

# generate new text
generated = generate_seq(model2, tokenizer2, seq_length2, seed_text2, 50)
print(generated)

se dio cuenta de que harry, ron, hermione y ginny lo miraban; los saludó con una seca cabezada y se dio la vuelta. así que ése es el pequeño scorpius murmuró ron. asegúrate de superarlo en todos los exámenes, rosie. suerte que has heredado la inteligencia de tu madre. haz el

dolor reconoció de que no había hecho que los menesteres la gente se ha quedado porque los sido las patas mientras la amor un poco detenerlo dijo harry harry se había utilizado de que los dos se había muerto la aparición harry y los mortífagos mientras se había muerto a


## Corpus 2 - Modelo 1

In [7]:
# load cleaned text sequences
in_filename = 'CodigoDaVinci_CleanText.txt'
doc3 = load_doc(in_filename)
lines3 = doc3.split('\n')
seq_length3 = len(lines3[0].split()) - 1

# load the model
model3 = load_model('model_DaVinci.h5')

# load the tokenizer
tokenizer3 = load(open('tokenizer_DaVinci.pkl', 'rb'))

In [8]:
# select a seed text
seed_text3 = lines3[randint(0,len(lines3))]
print(seed_text3 + '\n')

# generate new text
generated3 = generate_seq(model3, tokenizer3, seq_length3, seed_text3, 50)
print(generated3)

falso, o los siguientes serán sophie y usted. fue un accidente de coche intervino sophie, que notaba que el dolor de su infancia volvía a apoderarse de ella. ¡un accidente! cuentos infantiles para proteger su inocencia replicó teabing. piense que sólo dos miembros de la familia quedaron con vida, el gran

maestre como que se acercó más con la puerta que se lo la tumba de la puerta que se lo la tumba de la puerta que se lo londres en la mano y se le lo londres en la tumba de la puerta que se lo la tumba de la


## Corpus 2 - Modelo 2

In [9]:
# load cleaned text sequences
in_filename = 'CodigoDaVinci_CleanText.txt'
doc4 = load_doc(in_filename)
lines4 = doc4.split('\n')
seq_length4 = len(lines4[0].split()) - 1

# load the model
model4 = load_model('model2_DaVinci.h5')

# load the tokenizer
tokenizer4 = load(open('tokenizer2_DaVinci.pkl', 'rb'))

In [10]:
# select a seed text
seed_text4 = lines4[randint(0,len(lines4))]
print(seed_text4 + '\n')

# generate new text
generated4 = generate_seq(model4, tokenizer4, seq_length4, seed_text4, 50)
print(generated4)

la oscuridad. el brillo tenue y rojizo de las luces de emergencia apuntaba hacia arriba, iluminando con un resplandor artificial la colección de leonardos, tizianos y caravaggios suspendidos del techo con cables. naturalezas muertas, escenas religiosas y paisajes se alternaban con retratos de nobles y políticos. aunque la gran galería albergaba

el grial del grial no recordó que lo echarle un gesto en un reverso y que se había olvidado de que los ojos que se había dicho que había sido un momento y se había olvidado de cinco hurtos y los ojos y se ver que no había salido y
